# Data Cleaning

## Text Extraction

In [1]:
%pip install pandas requests beautifulsoup4 openpyxl

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re

# load the Excel file with URLs
df = pd.read_excel('web_scraping_urls.xlsx')

# function to clean text
def clean_text(text):
    # regex to match non-printable characters
    clean_text = re.sub(r'[\x00-\x1f\x7f-\x9f]', '', text)
    return clean_text

# function to extract and clean text from a URL
def extract_text(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        text = soup.get_text()
        # clean the text before returning
        return clean_text(text)
    except requests.RequestException as e:
        return str(e)

# define batch size and output file name
batch_size = 10
output_filename = 'all_extracted_texts.xlsx'

# initialize the ExcelWriter to append without overwriting previous data
with pd.ExcelWriter(output_filename, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
    total_urls = len(df['URLs'])

    # process each batch
    for start in range(0, total_urls, batch_size):
        end = start + batch_size
        print(f"processing batch from URL {start+1} to {min(end, total_urls)}")
        batch_data = {'URL': df['URLs'][start:end], 'Quarter': df['Quarter'][start:end], 'Extracted Text': []}

        for index, url in enumerate(df['URLs'][start:end], start=start+1):
            print(f"processing URL {index}/{total_urls}: {url}")
            text = extract_text(url)
            batch_data['Extracted Text'].append(text)
            time.sleep(2)  # Sleep for 2 seconds between requests
            print("extraction successful for this URL")

        # convert batch data to DataFrame and append to the Excel file
        batch_df = pd.DataFrame(batch_data)
        # append batch data to the Excel file
        batch_df.to_excel(writer, sheet_name='Extracted Text', index=False, header=not writer.sheets, startrow=writer.sheets['Extracted Text'].max_row if 'Extracted Text' in writer.sheets else 0)

        print(f"batch from URL {start+1} to {min(end, total_urls)} saved.")

print("all URLs processed and saved to the same Excel file.")


processing batch from URL 1 to 10
processing URL 1/4913: https://www.savills.com.sg/blog/article/216498/singapore-articles/emerging-trends-in-singapore-s-industrial-property-market.aspx#:~:text=With%20a%20slew%20of%20new,%25%20to%203%25%20in%202024.
extraction successful for this URL
processing URL 2/4913: https://www.colliers.com/en-sg/news/2024-02-14-q4-singapore-industrial-market-update-press#:~:text=Report%20highlights&text=The%20price%20index%20rose%20by,growth%20and%20weakness%20in%20exports.
extraction successful for this URL
processing URL 3/4913: https://www.businesstimes.com.sg/property/bt-property-week-2024/singapore-industrial-strong-growth-prospects-no-easy-ride
extraction successful for this URL
processing URL 4/4913: https://www.colliers.com/en-sg/research/2023-q4-singapore-industrial-report-colliers
extraction successful for this URL
processing URL 5/4913: https://www.jll.com.sg/en/newsroom/moderation-in-industrial-rent-and-price-growth-in-4q23
extraction successful for